# Day 9: Smoke Basin

[https://adventofcode.com/2021/day/9](https://adventofcode.com/2021/day/9)

## Description

### Part One

These caves seem to be [lava tubes](https://en.wikipedia.org/wiki/Lava_tube). Parts are even still volcanically active; small hydrothermal vents release smoke into the caves that slowly <span title="This was originally going to be a puzzle about watersheds, but we're already under water.">settles like rain</span>.

If you can model how the smoke flows through the caves, you might be able to avoid it and be that much safer. The submarine generates a heightmap of the floor of the nearby caves for you (your puzzle input).

Smoke flows to the lowest point of the area it's in. For example, consider the following heightmap:

    2199943210
    3987894921
    9856789892
    8767896789
    9899965678


Each number corresponds to the height of a particular location, where `9` is the highest and `0` is the lowest a location can be.

Your first goal is to find the _low points_ - the locations that are lower than any of its adjacent locations. Most locations have four adjacent locations (up, down, left, and right); locations on the edge or corner of the map have three or two adjacent locations, respectively. (Diagonal locations do not count as adjacent.)

In the above example, there are _four_ low points, all highlighted: two are in the first row (a `1` and a `0`), one is in the third row (a `5`), and one is in the bottom row (also a `5`). All other locations on the heightmap have some lower adjacent location, and so are not low points.

The _risk level_ of a low point is _1 plus its height_. In the above example, the risk levels of the low points are `2`, `1`, `6`, and `6`. The sum of the risk levels of all low points in the heightmap is therefore _`15`_.

Find all of the low points on your heightmap. _What is the sum of the risk levels of all low points on your heightmap?_

### Part Two

Next, you need to find the largest basins so you know what areas are most important to avoid.

A _basin_ is all locations that eventually flow downward to a single low point. Therefore, every low point has a basin, although some basins are very small. Locations of height `9` do not count as being in any basin, and all other locations will always be part of exactly one basin.

The _size_ of a basin is the number of locations within the basin, including the low point. The example above has four basins.

The top-left basin, size `3`:

    2199943210
    3987894921
    9856789892
    8767896789
    9899965678


The top-right basin, size `9`:

    2199943210
    3987894921
    9856789892
    8767896789
    9899965678


The middle basin, size `14`:

    2199943210
    3987894921
    9856789892
    8767896789
    9899965678


The bottom-right basin, size `9`:

    2199943210
    3987894921
    9856789892
    8767896789
    9899965678


Find the three largest basins and multiply their sizes together. In the above example, this is `9 * 14 * 9 = 1134`.

_What do you get if you multiply together the sizes of the three largest basins?_

In [1]:
from functools import reduce
from operator import mul


caves = []
with open('input') as input_file:
    for line in input_file:
        caves.append(list(map(int, line.strip())))


def get_max_point(caves):
    return len(caves[0]) - 1, len(caves) - 1


def get_adjacent_points(point, max_x, max_y):
    point_x, point_y = point
    for delta_x, delta_y in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        adj_x, adj_y = point_x + delta_x, point_y + delta_y
        if 0 <= adj_x <= max_x and 0 <= adj_y <= max_y:
            yield adj_x, adj_y


def get_low_points(caves):
    max_x, max_y = get_max_point(caves)
    for y in range(max_y + 1):
        for x in range(max_x + 1):
            for adj_x, adj_y in get_adjacent_points((x, y), max_x, max_y):
                if caves[adj_y][adj_x] <= caves[y][x]:
                    break
            else:
                yield x, y


def get_basin_size(caves, low_point):
    x, y = low_point
    if caves[y][x] >= 9:
        return 0
    caves[y][x] = 10
    adjacent_points = get_adjacent_points((x, y), *get_max_point(caves))
    return 1 + sum(get_basin_size(caves, adjacent_point) for adjacent_point in adjacent_points)


def get_basin_sizes(caves, low_points):
    for low_point in low_points:
        yield get_basin_size(caves, low_point)


low_points = list(get_low_points(caves))

print('Risk level sum:', sum(caves[y][x] + 1 for x, y in low_points))

basin_sizes = sorted(get_basin_sizes(caves, low_points), reverse=True)

print('Three largest basins product:', reduce(mul, basin_sizes[:3]))

Risk level sum: 475
Three largest basins product: 1092012
